# Navier Stokes Equation Reduced Basis Method 
Incompressible timedependet Navier Stokes equation

Find $(u,p):[0,T] \to (H_{0,D}^1)^d \times L^2$, s.t.

\begin{align}
\int_{\Omega} \partial_t u \cdot v + \int_{\Omega} \nu \nabla u \nabla v + u \cdot \nabla u v - \int_{\Omega} \operatorname{div}(v) p &= \int f v  && \forall v \in (H_{0,D}^1)^d, \\ 
- \int_{\Omega} \operatorname{div}(u) q &= 0 && \forall q \in L^2, \\
\quad u(t=0) & = u_0
\end{align}

In [1]:
from netgen import gui
from ngsolve import *
from netgen.geom2d import SplineGeometry
import scipy.sparse as sp
import numpy as np
from math import pi

In [2]:
from netgen.geom2d import SplineGeometry
geo = SplineGeometry()
geo.AddRectangle( (0, 0), (2, 0.41), bcs = ("wall", "outlet", "wall", "inlet"))
geo.AddCircle ( (0.2, 0.2), r=0.05, leftdomain=0, rightdomain=1, bc="cyl")
mesh = Mesh( geo.GenerateMesh(maxh=0.08))
mesh.Curve(3)
Draw(mesh)

In [3]:
# viscosity
nu = 0.001
dt = 0.001
dn = 1./dt
z = 100  # Number of snapshots

In [4]:
k = 3
V = VectorH1(mesh,order=k, dirichlet="wall|cyl|inlet")
Q = H1(mesh,order=k-1)
X = FESpace([V,Q])

In [5]:
gfu_h = GridFunction(X)
gfu_N = GridFunction(X)
gfu = GridFunction(X)
velocity = gfu.components[0]


uin = CoefficientFunction((1.5*4*y*(0.41-y)/(0.41*0.41),0))
gfu_h.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))

Draw(gfu_h.components[0],mesh,"gfu_h",sd=3)
Draw(gfu.components[0],mesh,"gfu",sd=3)

In [6]:
(u,p), (v,q) = X.TnT()

a = BilinearForm(X)
stokes = (nu*InnerProduct(grad(u),grad(v))-div(u)*q-div(v)*p)*dx
a += stokes
a.Assemble()

f = LinearForm(X)
f.Assemble()

inv_stokes = a.mat.Inverse(X.FreeDofs())

res = f.vec.CreateVector()
res.data = f.vec - a.mat*gfu_h.vec
gfu_h.vec.data += inv_stokes * res
gfu_N = gfu_h
Draw(gfu_h.components[0],mesh,"gfu_h",sd=3)

In [7]:
(u,p), (v,q) = X.TnT()

a = BilinearForm(X)
stokes = (nu*InnerProduct(grad(u),grad(v))-div(u)*q-div(v)*p)*dx
a += stokes
a.Assemble()

f = LinearForm(X)
f.Assemble()

inv_stokes = a.mat.Inverse(X.FreeDofs())

res = f.vec.CreateVector()
res.data = f.vec - a.mat*gfu.vec
gfu.vec.data += inv_stokes * res
Draw(gfu.components[0],mesh,"gfu",sd=3)

In [8]:
# matrix for implicit part of IMEX(1) scheme:
mstar = BilinearForm(X)

mstar += InnerProduct(u,v)*dx + dt*stokes

mstar.Assemble()

mh = BilinearForm(X)

mh += InnerProduct(u,v)*dx
mh.Assemble()

inv = mstar.mat.Inverse(X.FreeDofs())

In [9]:
M = BilinearForm(X)
M += InnerProduct(u,v)*dx
M.Assemble()

#D = BilinearForm(X)
#D += (div(u)*q+div(v)*p)*dx
#D.Assemble()

A = BilinearForm(X)
A += (nu*InnerProduct(grad(u),grad(v))-div(u)*q-div(v)*p)*dx
A.Assemble()

Mstar = BilinearForm(X)
Mstar += 2*InnerProduct(u,v)*dx + dt*(nu*InnerProduct(grad(u),grad(v))-div(u)*q-div(v)*p)*dx
Mstar.Assemble()
Inv_CN = Mstar.mat.Inverse(X.FreeDofs())

conv = BilinearForm(X, nonassemble=True)
conv += (grad(velocity)*velocity*v)*dx

unm1 = gfu.vec.CreateVector()
unm1 = gfu.vec



In [10]:
rows,cols,vals = A.mat.COO()
Ah = sp.csr_matrix((vals,(rows,cols)))

rows,cols,vals = M.mat.COO()
Mh = sp.csr_matrix((vals,(rows,cols)))

In [11]:
erg = np.zeros([len(gfu.vec),int(dn)+1])
S_erg = np.zeros([len(gfu.vec),z])
erg[:,0] = gfu.vec
V = np.zeros([len(gfu.vec),z]) # creat the orthogonal transformationsmatrix for the reducebasis method
line = np.linspace(0,dn-1,z) # stors the number of linear distributed number of iteration from the solution

### CNBA method 
For $n = \{0,1,2 ... N-1\}$

$
M^* = 2M + \nu k B -kD 
$

$
M \big(\frac{u^{n+1}-u^{n}}{k}\big) + C\big(\frac{3}{2}u^{n}-\frac{1}{2}u^{n-1}\big) + \nu B\big(\frac{1}{2}(u^{n+1}+u^{n})\big) - D\big(\frac{1}{2}(p^{n+1}+p^{n})\big) - D\big(\frac{1}{2}(u^{n+1}+u^{n})\big) = 0
$

$
u^{n+1}\big(2M+\nu kB -kD\big) = 2Mu^n - \nu kBu^n + kDu^n- 3kCu^n + kCu^{n-1}
$

where $M^* = 2M+\nu kB -kD$

$
M^*\big(u^{n+1} - u^{n} \big) = k\bigg[ u^n\big( 2D - 3C - 2\nu B\big) + Cu^{n-1} \bigg]
$

Where $u^{-1} := u_0$

In [12]:
V_sb = MultiVector(gfu.vec, 1)
V_sb[0] = gfu.vec
V[:,0] = np.array(V_sb[0])/np.linalg.norm(np.array(V_sb[0]))
erg[:,0] = gfu.vec
S_erg[:,0] = gfu.vec
o = 1
e = int(line[o])

for i in range(int(dn)):
    conv.Assemble()
    if i > 1:
        unm1.vec = erg[:,i-1]
    res.data =  3*conv.mat*gfu.vec + 2*A.mat*gfu.vec - conv.mat*unm1
    gfu.vec.data -= dt * Inv_CN * res
    
    for r in line:
        if int(r) == i: #desided which solution will be orthogonalirsed
            if i > 0:
                with TaskManager():
                    V_sb.AppendOrthogonalize(gfu.vec)
                V[:,o] = np.array(V_sb[o])
                S_erg[:,o] = gfu.vec
                o = o + 1
    
    velocity = gfu.components[0]
    erg[:,i+1] = gfu.vec
    Redraw()    
        

In [13]:
S_erg[0,0]

1.2476504924271326e-16

## POD method
A other way to get the transforamtions matrix is the POD method.

In [14]:
def POD(S):
    V = np.zeros([len(gfu.vec),z])
    
    #splitting the snapshotsolution like in the paper
    for i in range(S.shape[1]):
        S[:,i] = S[:,i] - 1/S.shape[0] * np.sum(S[:,i])
    
    if S.shape[1] <= S.shape[0]:
        C = np.transpose(S).dot(S)
        U, Sig, VH = np.linalg.svd(C, full_matrices=True)
        VH = np.transpose(VH)
        for i in range(C.shape[1]):
            V[:,i] = 1/np.sqrt(Sig[i]) * S.dot(VH[:,i])
        
        
    return V

In [23]:
V = POD(S_erg)
print(V[:,0].dot(V[:,0]))
print(len(V[:,0]))

1.000000000000001
4314


In [16]:
MN = np.transpose(V).dot(Mh.dot(V))
AN = np.transpose(V).dot(Ah.dot(V))

#conv_N = np.transpose(V).dot(conv_h.dot(V))

In [17]:
gfu_N = GridFunction(X)
gfu_N.vec.FV().NumPy()[:] = erg[:,0] 
gfu.vec.FV().NumPy()[:] = erg[:,0] 
uN = np.transpose(V).dot(erg[:,0])
velocity_N = gfu_N.components[0]
Draw(gfu_N.components[0],mesh,"gfu_N",sd=3)

In [18]:
err = res.CreateVector()
Draw(gfu_N.components[0]-gfu.components[0],mesh,"error",sd=3)

$
\bigg(\frac{Id}{\Delta t} + \frac{1}{2}\nu B \bigg) a^{n+1} = a^n - \bigg( C_1 \tilde{a}^{n-\frac{1}{2}} + \frac{1}{2}\nu B a^n\bigg)
$

$
M^* = \frac{Id}{\Delta t} + \frac{1}{2}\nu B
$

$
M^*\bigg(a^{n+1} - a^{n}\bigg) = - \frac{1}{2}\nu B a^{n+1} - C_1 \tilde{a}^{n-\frac{1}{2}}
$

In [24]:
C1ij = BilinearForm(X, nonassemble=True)
C1ij += (grad(velocity_N)*velocity_N*v)*dx

TypeError: __mul__(): incompatible function arguments. The following argument types are supported:
    1. (self: ngsolve.fem.CoefficientFunction, cf: ngsolve.fem.CoefficientFunction) -> ngsolve.fem.CoefficientFunction
    2. (self: ngsolve.fem.CoefficientFunction, value: float) -> ngsolve.fem.CoefficientFunction
    3. (self: ngsolve.fem.CoefficientFunction, value: complex) -> ngsolve.fem.CoefficientFunction
    4. (self: ngsolve.fem.CoefficientFunction, arg0: ngfem::DifferentialSymbol) -> ngfem::SumOfIntegrals

Invoked with: <ngsolve.comp.GridFunctionCoefficientFunction object at 0x7f3b39b74170>, array([-3.16722105e-06, -3.16722105e-06, -3.16722105e-06, ...,
        5.08758889e-03,  2.91401263e-03,  1.25292191e-03])

In [27]:
print(v.vec)

AttributeError: 'ngsolve.comp.ProxyFunction' object has no attribute 'vec'

In [20]:
C0 = LinearForm(X)
C0 += InnerProduct(grad(velocity_N)*velocity_N,v)*dx

In [21]:
Residium = []
time = []
erg_N = np.zeros([len(gfu_N.vec),int(dn)+1])
I = np.eye(z, dtype=int)

erg_N[:,0] = gfu_N.vec

MN_star = I + 0.5*dt*AN
inv_N = np.linalg.inv(MN_star)
for i in range(int(dn)):
    err.vec = sqrt (Integrate ((gfu.components[0]-gfu_N.components[0])*(gfu.components[0]-gfu_N.components[0]),mesh))
    Residium.append(err.vec)
    C0.Assemble()
    C0_N = np.transpose(V).dot(C0.vec)
    res_N = -0.5*dt*AN.dot(uN) - C0_N
    uN -= uN + dt * inv_N.dot(res_N)
    gfu.vec.FV().NumPy()[:] = erg[:,i+1]
    velocity_N = gfu_N.components[0]
    gfu_N.vec.FV().NumPy()[:] = (uN).dot(np.transpose(V))
    erg_N[:,i+1] = gfu_N.vec
    time.append(dt*i)
    velocity_N = gfu_N.components[0]
    Redraw()

In [ ]:
C1ij.mat

In [ ]:
C0 = LinearForm(X)
C0 += (grad(velocity_N)*velocity_N*v)*dx

In [ ]:
Residium = []
time = []
erg_N = np.zeros([len(gfu_N.vec),int(dn)+1])
erg_N[:,0] = gfu_N.vec
unm1_N = np.transpose(V).dot(erg_N[:,0])
MN_star = 2*MN + dt*AN
inv_N = np.linalg.inv(MN_star)
for i in range(int(dn)):
    err.vec = sqrt (Integrate ((gfu.components[0]-gfu_N.components[0])*(gfu.components[0]-gfu_N.components[0]),mesh))
    Residium.append(err.vec)
    C0.Assemble()
    C0_N = np.transpose(V).dot(C0.vec)
    
    
    if i > 1:
        unm1_N = np.transpose(V).dot(erg_N[:,i-1])
        
    #res.data =  3*conv.mat*gfu.vec + 2*A.mat*gfu.vec - conv.mat*unm1
    #gfu.vec.data -= dt * Inv_CN * res
    
    res_N =  3*C0_N.dot(uN) + 2*AN.dot(uN) - C0_N.dot(unm1_N)
    uN -= dt * inv_N.dot(res_N)

    gfu.vec.FV().NumPy()[:] = erg[:,i+1]
    
    gfu_N.vec.FV().NumPy()[:] = (uN).dot(np.transpose(V))
    velocity_N = gfu_N.components[0]
    erg_N[:,i+1] = gfu_N.vec
    time.append(dt*i)
    
   
    Redraw()

In [ ]:
import matplotlib.pyplot as plt
plt.title("Total error {} dt {}".format(z,dt))
plt.ylabel("error")
plt.xlabel("time [s]")
plt.grid()
plt.semilogy(time,Residium)
plt.savefig("plots/Total error {} dt POD".format(z))

In [31]:
D = np.array([[2,2],[2,2]])
print(np.outer(1,2))

[[2]]
